# Training

### Imports

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import preprocessing

from settings import *
import utils
from callbacks import *
from gan import GAN

### Check GPU

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")

if gpus:

	try:
		tf.config.experimental.set_visible_devices(gpus[0], "GPU")
		print("Using GPU :)")

	except RuntimeError as e:
		print(e)

else:
	print("Using CPU :(")

Using GPU :)


### Dataset

In [3]:
dataset = preprocessing.image_dataset_from_directory(
	DATA_DIR,
	label_mode = None,
	color_mode = "rgb",
	batch_size = BATCH_SIZE,
	image_size = (IMAGE_SIZE, IMAGE_SIZE),
	shuffle = True
)

dataset = dataset.map(utils.tf_norm_img)

if FLIP_DATASET:
	flipped_dataset = dataset.map(tf.image.flip_left_right)
	dataset = dataset.concatenate(flipped_dataset)
	dataset = dataset.shuffle(BATCH_SIZE)

print("Dataset final size:", NB_DATA)

Found 40603 files belonging to 1 classes.
Dataset final size: 81206


### Model

In [4]:
gan = GAN()
gan.compile()
gan.summary()

Mapping: (None, 512) -> (None, 512) | 2101248 parameters
Generator: [(None, 1), (None, 512) * 6, (None, 128, 128, 1) * 11] -> (None, 128, 128, 3) | 17157085 parameters
Discriminator: (None, 128, 128, 3) -> (None, 1) | 16438401 parameters


### First run / Continue

In [5]:
if not os.path.exists(SAMPLES_DIR):
	os.makedirs(SAMPLES_DIR)

if not os.path.exists(MODELS_DIR):
	os.makedirs(MODELS_DIR)

utils.reset_rand()
initial_epoch = gan.load_weights(MODELS_DIR)

if initial_epoch == 0:
	samples_z = np.random.normal(0., 1., (OUTPUT_SHAPE[0] * OUTPUT_SHAPE[1], LATENT_DIM))
	samples_noise = np.random.normal(0., 1., ((NB_BLOCKS * 2) - 1, OUTPUT_SHAPE[0] * OUTPUT_SHAPE[1], IMAGE_SIZE, IMAGE_SIZE, 1))
	np.save(os.path.join(OUTPUT_DIR, "samples_z.npy"), samples_z)
	np.save(os.path.join(OUTPUT_DIR, "samples_noise.npy"), samples_noise)

else:
	samples_z = np.load(os.path.join(OUTPUT_DIR, "samples_z.npy"))
	samples_noise = np.load(os.path.join(OUTPUT_DIR, "samples_noise.npy"))

### Training

In [6]:
history = gan.fit(
	dataset,
	batch_size = BATCH_SIZE,
	epochs = NB_EPOCHS,
	shuffle = True,
	initial_epoch = initial_epoch,
	callbacks = [
		Updates(),
		SaveSamples(samples_z, samples_noise),
		SaveModels()
	]
)

Epoch 10/1000
  311/10152 [..............................] - ETA: 1:00:23 - Generator loss: 1.7044 - Discriminator loss: 0.7587 - Gradient penalty: 0.2061